# Assigment 5

**Submission deadlines**:

* last lab before 27.06.2022 

**Points:** Aim to get 12 out of 15+ possible points

All needed data files are on Drive: <https://drive.google.com/drive/folders/1uufpGn46Mwv4oBwajIeOj4rvAK96iaS-?usp=sharing> (or will be soon :) )

## Task 1 (5 points)

Consider the vowel reconstruction task -- i.e. inserting missing vowels (aeuioy) to obtain proper English text. For instance for the input sentence:

<pre>
h m gd smbd hs stln ll m vwls
</pre>

the best result is

<pre>
oh my god somebody has stolen all my vowels
</pre>

In this task both dev and test data come from the two books about Winnie-the-Pooh. You have to train two RNN Language Models on *pooh-train.txt*. For the first model use the code below, for the second choose different hyperparameters (different dropout, smaller number of units or layers, or just do any modification you want). 

The code below is based on
https://www.kdnuggets.com/2020/07/pytorch-lstm-text-generation-tutorial.html

In [1]:
from collections import Counter

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

SEQUENCE_LENGTH = 15

class PoohDataset(torch.utils.data.Dataset):
    def __init__(self, sequence_length, device):
        txt = open('pooh_train.txt').read()
        
        self.words = txt.lower().split() # The text is already tokenized
        
        self.uniq_words = self.get_uniq_words()

        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        self.words_indexes = [self.word_to_index[w] for w in self.words]
        self.sequence_length = sequence_length
        self.device = device


    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.words_indexes) - self.sequence_length

    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+self.sequence_length], device=self.device),
            torch.tensor(self.words_indexes[index+1:index+self.sequence_length+1], device=self.device)
        )
        
pooh_dataset = PoohDataset(SEQUENCE_LENGTH, device)        

In [2]:
from torch import nn


class LSTMModel(nn.Module):
    def __init__(self, dataset, device):
        super(LSTMModel, self).__init__()
        self.lstm_size = 512
        self.embedding_dim = 100
        self.num_layers = 2
        self.device = device
        

        n_vocab = len(dataset.uniq_words)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)
        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size).to(self.device),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size).to(self.device))
        
model = LSTMModel(pooh_dataset, device) 
model.to(device)

LSTMModel(
  (embedding): Embedding(2548, 100)
  (lstm): LSTM(100, 512, num_layers=2, dropout=0.2)
  (fc): Linear(in_features=512, out_features=2548, bias=True)
)

In [3]:
import numpy as np
from torch.utils.data import DataLoader

batch_size = 512
max_epochs = 30

def train(dataset, model):
    model.train()

    dataloader = DataLoader(dataset, batch_size=batch_size)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(max_epochs):
        state_h, state_c = model.init_state(SEQUENCE_LENGTH)
        
        for batch, (x, y) in enumerate(dataloader):
            
            optimizer.zero_grad()

            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()            

            loss.backward()
            optimizer.step()

        print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })

In [ ]:
train(pooh_dataset, model)

In [ ]:
torch.save(model.state_dict(), 'pooh_2x512_30ep.model')

In [5]:
model.load_state_dict(torch.load('pooh_2x512_30ep.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [6]:
# You can use the code if you want

from collections import defaultdict as dd

vowels = set("aoiuye'")
def devowelize(s):
    rv = ''.join(a for a in s if a not in vowels)
    if rv:
        return rv
    return '_' # Symbol for words without consonants

pooh_words = set(open('pooh_words.txt').read().split())
representation = dd(set)

for w in pooh_words:
    r = devowelize(w)
    representation[r].add(w)
    
hard_words = set()
for r, ws in representation.items():
    if len(ws) > 1:
        hard_words.update(ws)
        
print (len(hard_words))

863


In [7]:
def get_matching_logits(logits, word, dataset):
    keys = list(representation[word])
    values = []
    for reconstructed_word in keys:
        if reconstructed_word in dataset.word_to_index:
            values.append(logits[dataset.word_to_index[reconstructed_word]].item())
        else:
            values.append(min(logits))
    return keys, values

In [8]:
original_text = open('pooh_test.txt').read().lower().split()
devowelized_text = [devowelize(w) for w in original_text]

In [11]:
# the predict function is a text generator. you have to modify this code!

def predict(dataset, model, words, next_words, prev_range=5, temperature=1):
    model.eval()

    state_h, state_c = model.init_state(len(words))

    probability = 0

    previous_indices = []
    for word in words:
        if word in dataset.word_to_index:
            previous_indices.append(dataset.word_to_index[word])

    for i in range(0, len(next_words)):
        x = torch.tensor([previous_indices[-prev_range:]])
        x = x.to(device)

        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        matching_words, matching_logits = get_matching_logits(last_word_logits, next_words[i], dataset)
        p = nn.functional.softmax(torch.tensor(matching_logits)/temperature, dim=0).detach().cpu().numpy()
        word_index = np.random.choice(len(matching_logits), p=p)
        chosen_word = matching_words[word_index]
        words.append(chosen_word)
        if chosen_word in dataset.word_to_index:
            previous_indices.append(dataset.word_to_index[chosen_word])

        probability += np.log(p[word_index])

    return ' '.join(words), probability

In [19]:
def accuracy(original_sequence, reconstructed_sequence):
    sa = original_sequence
    sb = reconstructed_sequence
    score = len([1 for (a,b) in zip(sa, sb) if a == b])
    return score / len(original_sequence)


## Random

In [108]:
random_prediction = []
for word in devowelized_text:
    random_prediction.append(np.random.choice(list(representation[word])))

In [110]:
accuracy(original_text, random_prediction)

0.5365480290853425

## Model 1

In [21]:
prev_range = 3
prompt = original_text[:prev_range]
next_words = devowelized_text[prev_range:]
prediction, _ = predict(pooh_dataset, model, prompt, next_words, prev_range=prev_range)

In [22]:
accuracy(original_text, prediction.split())

0.8413062890674831

In [ ]:
best_prediction = None
best_log_prob = -np.inf

prev_range = 3
for _ in range(10):
    prompt = original_text[:prev_range]
    next_words = devowelized_text[prev_range:]
    prediction, log_prob = predict(pooh_dataset, model, prompt, next_words, prev_range=prev_range)
    if log_prob > best_log_prob:
        best_prediction = prediction
        best_log_prob = log_prob

In [20]:
accuracy(original_text, best_prediction.split())

0.8423268274014543

## Model 2

In [23]:
class LSTMModel(nn.Module):
    def __init__(self, dataset, device):
        super(LSTMModel, self).__init__()
        self.lstm_size = 256
        self.embedding_dim = 50
        self.num_layers = 2
        self.device = device


        n_vocab = len(dataset.uniq_words)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)
        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size).to(self.device),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size).to(self.device))

model = LSTMModel(pooh_dataset, device)
model.to(device)

LSTMModel(
  (embedding): Embedding(2548, 50)
  (lstm): LSTM(50, 256, num_layers=2, dropout=0.2)
  (fc): Linear(in_features=256, out_features=2548, bias=True)
)

In [26]:
model.load_state_dict(torch.load('pooh_2x256_30ep.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [27]:
prev_range = 3
prompt = original_text[:prev_range]
next_words = devowelized_text[prev_range:]
prediction, _ = predict(pooh_dataset, model, prompt, next_words, prev_range=prev_range)

In [28]:
accuracy(original_text, prediction.split())

0.8405408853170048

In [29]:
best_prediction = None
best_log_prob = -np.inf

prev_range = 3
for _ in range(10):
    prompt = original_text[:prev_range]
    next_words = devowelized_text[prev_range:]
    prediction, log_prob = predict(pooh_dataset, model, prompt, next_words, prev_range=prev_range)
    if log_prob > best_log_prob:
        best_prediction = prediction
        best_log_prob = log_prob


In [30]:
accuracy(original_text, best_prediction.split())

0.840285750733512

You can assume that only words from pooh_words.txt can occur in the reconstructed text. For decoding you have two options (choose one, or implement both ang get **+1** bonus point)

1. Sample reconstructed text several times (with quite a low temperature), choose the most likely result.
2. Perform beam search.

Of course in the sampling procedure you should consider only words matching the given consonants.

Report accuracy of your methods (for both language models). The accuracy should be computed by the following function, it should be *greater than 0.25*.


```python
def accuracy(original_sequence, reconstructed_sequence):
    sa = original_sequence
    sb = reconstructed_sequence
    score = len([1 for (a,b) in zip(sa, sb) if a == b])
    return score / len(original_sequence)
```


## Task 2 (6 points)

This task is about text generation. You have to:

**A**. Create text corpora containing texts with similar vocabulary (for instance books from the same genre, or written by the same author). This corpora should have approximately 1M words. You can consider using the following sources: Project Gutenberg (https://www.gutenberg.org/), Wolne Lektury (https://wolnelektury.pl/), parts of BookCorpus, https://github.com/soskek/bookcorpus, but generally feel free. Texts could be in English, Polish or any other language you know.

**B**. choose the tokenization procedure. It should have two stages:

1. word tokenization (you can use nltk.tokenize.word_tokenize, tokenizer from spaCy, pytorch, keras, ...). Test your tokenizer on your corpora, and look at a set of tokens containing both letters and special characters. If some of them should be in your opinion treated as a sequence of tokens, then modify the tokenization procedure

2. sub-word tokenization (you can either use the existing procedure, like wordpiece or sentencepiece, or create something by yourself). Here is a simple idea: take 8K most popular words (W), 1K most popular suffixes (S), and 1K most popular prefixes (P). Words in W are its own tokens. Word x outside W should be tokenized as 'p_ _s' where p is the longest prefix of x in P, and s is the longest prefix of W

**C**. write text generation procedure. The procedure should fulfill the following requirements:

1. it should use the RNN language model (trained on sub-word tokens)
2. generated tokens should be presented as a text containing words (without extra spaces, or other extra characters, as begin-of-word introduced during tokenization)
3. all words in a generated text should belond to the corpora (note that this is not guaranteed by LSTM)
4. in generation Top-P sampling should be used (see NN-NLP.6, slide X) 
5. in generated texts every token 3-gram should be uniq
6. *(optionally, +1 point)* all token bigrams in generated texts occur in the corpora

In [1]:
import re

import numpy as np
import torch

from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers
from torch.utils.data import DataLoader
from torch import nn, optim
from tokenizers import BertWordPieceTokenizer, Tokenizer

In [2]:
book_numbers = [2097, 834, 139, 48320, 108, 3289, 2350, 903, 537, 3070, 2347]

In [3]:
books = [strip_headers(load_etext(book_number)) for book_number in book_numbers]

In [4]:
sum([len(book.split()) for book in books])

840474

In [4]:
for n, book in enumerate(books):
    with open(f'book_{n}.txt', 'w') as f_out:
        f_out.write(book)

In [41]:
tokenizer = BertWordPieceTokenizer()

In [42]:
tokenizer.train_from_iterator(books, vocab_size=10000)

In [5]:
class Node:
    def __init__(self):
        self.edges = {}
        self.is_end = False

    def next_tokens(self):
        return list(self.edges.keys())

class Trie:
    def __init__(self):
        self.root = Node()

    def insert(self, tokens):
        current_node = self.root
        for token in tokens:
            if token not in current_node.edges:
                current_node.edges[token] = Node()
            current_node = current_node.edges[token]
        current_node.is_end = True

In [6]:
tokenizer = Tokenizer.from_file('tokenizer.json')


In [8]:
def ids_to_tokens(token_ids):
    return [tokenizer.id_to_token(token_id) for token_id in token_ids]

In [9]:
words = set()
for book in books:
    encoded = tokenizer.encode(book)
    transformed_text = tokenizer.decode(encoded.ids)
    words |= set(re.split('\W+', transformed_text))


In [10]:
used_ids = set()
for word in words:
    used_ids |= set(tokenizer.encode(word).ids)

In [11]:
all_ids = set(tokenizer.get_vocab().values())

In [12]:
special_ids = []
patt = re.compile('#*\w+')
for token_id in (all_ids - used_ids):
    token = tokenizer.id_to_token(token_id)
    if not patt.match(token):
       special_ids.append(token_id)

In [13]:
ids_to_tokens(special_ids)

['[PAD]',
 '[UNK]',
 '[CLS]',
 '[SEP]',
 '[MASK]',
 '!',
 '"',
 '$',
 '&',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '>',
 '?',
 '[',
 ']',
 '{',
 '}',
 '£',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '″',
 '・',
 '£1']

In [14]:
trie = Trie()

In [15]:
for word in words:
    trie.insert(tokenizer.encode(word).ids)

for id in special_ids:
    trie.insert([id])

In [16]:
class DoyleDataset(torch.utils.data.Dataset):
    def __init__(self, file, tokenizer, sequence_length, device):
        txt = open(file).read()

        self.encoded = tokenizer.encode(txt)
        self.word_indices = self.encoded.ids

        self.sequence_length = sequence_length
        self.device = device


    def __len__(self):
        return len(self.word_indices) - self.sequence_length

    def __getitem__(self, index):
        return (
            torch.tensor(self.word_indices[index:index+self.sequence_length], device=self.device),
            torch.tensor(self.word_indices[index+1:index+self.sequence_length+1], device=self.device)
        )


In [17]:
dataset = DoyleDataset('book_0.txt', tokenizer, 15, 'cpu')

In [18]:

class LSTMModel(nn.Module):
    def __init__(self, tokenizer, device):
        super(LSTMModel, self).__init__()
        self.lstm_size = 512
        self.embedding_dim = 100
        self.num_layers = 2
        self.device = device

        self.tokenizer = tokenizer

        n_vocab = self.tokenizer.get_vocab_size()
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)
        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size).to(self.device),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size).to(self.device))

    def fit(self, datasets, batch_size=512, max_epochs=30):
        self.train()

        dataloaders = [DataLoader(dataset, batch_size=batch_size) for dataset in datasets]

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        for epoch in range(max_epochs):
            state_h, state_c = model.init_state(SEQUENCE_LENGTH)

            for dataloader in dataloaders:
                for batch, (x, y) in enumerate(dataloader):

                    optimizer.zero_grad()

                    y_pred, (state_h, state_c) = model(x, (state_h, state_c))
                    loss = criterion(y_pred.transpose(1, 2), y)

                    state_h = state_h.detach()
                    state_c = state_c.detach()

                    loss.backward()
                    optimizer.step()

            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })




In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEQUENCE_LENGTH = 15

In [20]:
# model = LSTMModel(tokenizer, device)
# model.to(device)
#
# datasets = [DoyleDataset(f'book_{n}.txt', tokenizer, SEQUENCE_LENGTH, device) for n in range(11)]
# model.fit(datasets)
# torch.save(model.state_dict(), 'doyle.model')

In [21]:
model = LSTMModel(tokenizer, device)
model.to(device)
model.load_state_dict(torch.load('doyle.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [23]:
from collections import defaultdict

In [67]:
def top_p_sample(logits, temperature, p):
    probabilities = softmax(torch.tensor(logits, dtype=torch.int)/temperature, dim=0).detach().cpu().numpy()
    indices = np.argsort(probabilities)[::-1]
    probabilities = probabilities[indices]
    k = np.searchsorted(np.cumsum(probabilities), p) + 1

    probabilities = probabilities[:k] / np.sum(probabilities[:k])

    index = np.random.choice(indices[:k], p=probabilities)

    return index

In [68]:
from torch.nn.functional import softmax

def predict(model, prompt, tokens_to_generate, prev_range=5, temperature=0.7, p=0.9):
    model.eval()

    tokens = model.tokenizer.encode(prompt).ids
    state_h, state_c = model.init_state(len(tokens))

    current_node = trie.root

    unique_3_grams = defaultdict(list)

    for i in range(0, tokens_to_generate):
        x = torch.tensor([tokens[-prev_range:]])
        x = x.to(device)

        if current_node.is_end:
            allowed_tokens = np.union1d(current_node.next_tokens(), trie.root.next_tokens())
        else:
            allowed_tokens = np.array(current_node.next_tokens())

        allowed_tokens = allowed_tokens.astype(int)

        allowed_tokens = np.setdiff1d(allowed_tokens, unique_3_grams[tuple(tokens[-2:])])

        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        allowed_logits = last_word_logits.detach().numpy()[allowed_tokens]
        index = top_p_sample(allowed_logits, temperature, p)
        token_id = allowed_tokens[index]

        if token_id in current_node.next_tokens():
            current_node = current_node.edges[token_id]
        else:
            current_node = trie.root.edges[token_id]

        unique_3_grams[tuple(tokens[-2:])].append(token_id)
        tokens.append(token_id)

    return model.tokenizer.decode(tokens)


In [79]:
predict(model, "On reaching London I drove", 500)

'on reaching london i drove a first time before, so as to see it. the whole business, however, was more genuine than ever, and the incident gave a case. " yes, i am a hard, " said he. " you have had uneasy, watson, we shall have a word. " " my dear fellow, you are not yourself. " holmes leaned back in his chair. " oh, my dear watson, i have no doubt that you will know that i have said this very hour. " i had a long way of his heart. " here\'s a friend! " said i. " my god! " i cried. " what do you think of the attempted? ” “ i can give you a sketch of the name. " the baronet\'s voice was not unlike a hound. " we\'ve been in the bed. " he spoke in a feeble voice, but he was at the back of his voice. " now, watson. i will watch you. " “ i must confess that i would have you for your future acquiescence. " chapter 14 the hound of the baskervilles holmes. " then he lied to me, holmes. i could not forget the manager of his head. but the instant i tried to impress him out in a cold face, uncer

In [74]:
predict(model, "i turned my lantern down", 500)

'i turned my lantern down the the the of four. mortimer\'s, of the stapletons, of merripit house. a cold sweat bathed from the influence of paper upon the table. holmes sprang out of bed, but on the passage he sprang out upon his hands. " you can understand, watson, that you have not aided me, " she said. " and leave you to see that the disguise of yours is suggested by the\'80. " " i\'ll help you, watson. you will find your own nerves\'s work. " holmes leaned forward and laid it open. " thank you. " he looked down at my companion. " what about this, holmes? ” he asked. the old gentleman nodded. " why should i demonstrate you? " " yes, i have had a few weeks. " she glanced at me in amazement. " well, then, you can do, " holmes answered. " i give you my word that you will have a good deal to worry. " the same thought became intent. " a nice villain! " she cried. " it is a very curious world. " his eyes shone upon me. " my word, holmes, " he whispered. " that is your letter. i\'ve had a 

In [75]:
predict(model, "i turned my lantern down", 500)

'i turned my lantern down as he cried i heard the striking silver of my revolver. " you will excuse me, watson. you have no respect for me. " " i can hardly imagine, " i explained. " well, i shouldn\'t be surprised, madam. we can only have my case in any case. but you are a visitor, dr. watson, but you will see the situation. " holmes groaned. " get your distance by any means. i don\'t know! " " a good deal. " the fog of the fanlight had been a case. " my word! " he cried. " but why? " " ah, you will know that it is difficult. you can drive a savage, fixed yourself, and you have to fear. " i could see the gleam of keys and of furniture. " a hound, watson, " he said. " is he delirious? " holmes looked up and glanced keenly into my chair. " now, watson! " she cried. i could not help him. " oh, it is painful to have been, but it is not my duty. it is time that i have spoken to you. i will promise you that it will not be the condition. if you will persuade me to get down to - morrow mornin

## Task 3

In this task you have to create a network which looks at characters of the word and tries to guess whether the word is a noun, a verb, an adjective, and so on. To be more precise: the input is a word (without context), the output is a POS-tag (Part-of-Speech). Since some words are unambiguous, and we have no context, our network is supposed to return the set of possible tags.

The data is taken from Universal Dependencies English corpus, and of course it contains errors, especially because not all possible tags occured in the data.

Train a network (4p) or two networks (+2p) solving this task. Both networks should look at character n-grams occuring in the word. There are two options:

* **Fixed size:** for instance take 2,3, and 4-character suffixes of the word, use them as  features (whith 1-hot encoding). You can also combine prefix and suffix features. Simple, useful trick: when looking at suffixes, add some '_' characters at the beginning of the word to guarantee that shorter words have suffixes of a desired length.

* **Variable size:** take for instance 4-grams (or 4 grams and 3-grams), use Deep Averaging Network. Simple trick: add extra character at the beginning and at the end of the word, to add the information, that ngram occurs at special position ('ed' at the end has slightly different meaning that 'ed' in the middle)


## Task 4

Apply seq2seq model (you can modify the code from this tutorial: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) to compute grapheme to phoneme conversion for English. Train the model on dev_cmu_dict.txt and test it on test_cmu_dict.txt. Report accuracy of your solution using two metrics:
* exact match (how many words are perfectly converted to phonemes)
* exact match without stress (how many words are perfectly converted to phonemes when we remove the information about stress)
